# BPNN Model for EKG Classification

This notebook implements a Backpropagation Neural Network (BPNN) with complete mathematical formulations for EKG/ECG signal classification.

## Mathematical Foundations of BPNN

### 1. Forward Propagation

For a neural network with layers $l = 1, 2, ..., L$:

#### Linear Transformation
$$z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$$

where:
- $W^{[l]}$ is the weight matrix for layer $l$
- $a^{[l-1]}$ is the activation from previous layer
- $b^{[l]}$ is the bias vector
- $z^{[l]}$ is the pre-activation output

#### Activation Function
$$a^{[l]} = g^{[l]}(z^{[l]})$$

### 2. Activation Functions

#### Sigmoid
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

$$\sigma'(z) = \sigma(z)(1 - \sigma(z))$$

#### Hyperbolic Tangent (tanh)
$$\tanh(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$

$$\tanh'(z) = 1 - \tanh^2(z)$$

#### ReLU (Rectified Linear Unit)
$$\text{ReLU}(z) = \max(0, z)$$

$$\text{ReLU}'(z) = \begin{cases} 1 & \text{if } z > 0 \\ 0 & \text{if } z \leq 0 \end{cases}$$

#### Softmax (for output layer)
$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$$

### 3. Loss Functions

#### Binary Cross-Entropy
$$L(y, \hat{y}) = -\frac{1}{m} \sum_{i=1}^{m} [y^{(i)} \log(\hat{y}^{(i)}) + (1-y^{(i)}) \log(1-\hat{y}^{(i)})]$$

#### Categorical Cross-Entropy
$$L(y, \hat{y}) = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_k^{(i)} \log(\hat{y}_k^{(i)})$$

#### Mean Squared Error
$$L(y, \hat{y}) = \frac{1}{2m} \sum_{i=1}^{m} (y^{(i)} - \hat{y}^{(i)})^2$$

### 4. Backpropagation

#### Output Layer Gradient
$$\delta^{[L]} = \frac{\partial L}{\partial z^{[L]}} = a^{[L]} - y$$

#### Hidden Layer Gradient
$$\delta^{[l]} = (W^{[l+1]})^T \delta^{[l+1]} \odot g'^{[l]}(z^{[l]})$$

where $\odot$ denotes element-wise multiplication.

#### Weight Gradients
$$\frac{\partial L}{\partial W^{[l]}} = \frac{1}{m} \delta^{[l]} (a^{[l-1]})^T$$

#### Bias Gradients
$$\frac{\partial L}{\partial b^{[l]}} = \frac{1}{m} \sum_{i=1}^{m} \delta^{[l]}_i$$

### 5. Weight Update Rules

#### Gradient Descent
$$W^{[l]} := W^{[l]} - \alpha \frac{\partial L}{\partial W^{[l]}}$$
$$b^{[l]} := b^{[l]} - \alpha \frac{\partial L}{\partial b^{[l]}}$$

where $\alpha$ is the learning rate.

#### Momentum
$$v_{dW} = \beta v_{dW} + (1-\beta) dW$$
$$W := W - \alpha v_{dW}$$

#### Adam Optimizer
$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$$
$$\hat{m}_t = \frac{m_t}{1-\beta_1^t}$$
$$\hat{v}_t = \frac{v_t}{1-\beta_2^t}$$
$$\theta_t = \theta_{t-1} - \frac{\alpha}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import json

# Set random seed for reproducibility
np.random.seed(42)

## Synthetic EKG Dataset Generation

### Mathematical Model for EKG Synthesis

We model EKG signals as a combination of Gaussian functions representing PQRST complex:

$$ECG(t) = \sum_{i \in \{P, Q, R, S, T\}} A_i \exp\left(-\frac{(t - t_i)^2}{2\sigma_i^2}\right)$$

where $A_i$ is amplitude, $t_i$ is time position, and $\sigma_i$ is width of each wave component.

In [ ]:
def generate_pqrst_complex(heart_rate=70, sampling_rate=250, duration=1.0, noise_level=0.05):
    """
    Generate synthetic EKG signal with PQRST complex.
    ECG(t) = Σ A_i * exp(-(t - t_i)² / (2σ_i²))
    
    Parameters:
    -----------
    heart_rate : int
        Heart rate in beats per minute
    sampling_rate : int
        Samples per second
    duration : float
        Duration in seconds
    noise_level : float
        Standard deviation of Gaussian noise
    
    Returns:
    --------
    signal : numpy.ndarray
        Synthetic EKG signal
    """
    n_samples = int(duration * sampling_rate)
    t = np.linspace(0, duration, n_samples)
    signal = np.zeros(n_samples)
    
    # Calculate beat period
    beat_period = 60.0 / heart_rate
    n_beats = int(duration / beat_period)
    
    # PQRST wave parameters (time offset, amplitude, width)
    waves = {
        'P': (0.16, 0.25, 0.02),   # P wave
        'Q': (0.20, -0.15, 0.01),  # Q wave
        'R': (0.22, 1.5, 0.015),   # R wave (peak)
        'S': (0.24, -0.4, 0.01),   # S wave
        'T': (0.38, 0.35, 0.04),   # T wave
    }
    
    # Generate each heartbeat
    for beat in range(n_beats + 1):
        beat_start = beat * beat_period
        
        for wave_name, (t_offset, amplitude, width) in waves.items():
            wave_center = beat_start + t_offset
            # Gaussian function: A * exp(-(t - t_i)² / (2σ²))
            wave = amplitude * np.exp(-((t - wave_center) ** 2) / (2 * width ** 2))
            signal += wave
    
    # Add baseline wander (low-frequency noise)
    baseline = 0.1 * np.sin(2 * np.pi * 0.2 * t)
    
    # Add high-frequency noise
    noise = noise_level * np.random.randn(n_samples)
    
    return signal + baseline + noise

def generate_abnormal_ekg(condition='tachycardia', sampling_rate=250, duration=1.0):
    """
    Generate abnormal EKG patterns.
    
    Parameters:
    -----------
    condition : str
        Type of abnormality: 'tachycardia', 'bradycardia', 'arrhythmia'
    sampling_rate : int
        Samples per second
    duration : float
        Duration in seconds
    
    Returns:
    --------
    signal : numpy.ndarray
        Abnormal EKG signal
    """
    if condition == 'tachycardia':
        # Fast heart rate (>100 bpm)
        hr = np.random.randint(100, 140)
        return generate_pqrst_complex(hr, sampling_rate, duration, noise_level=0.08)
    
    elif condition == 'bradycardia':
        # Slow heart rate (<60 bpm)
        hr = np.random.randint(40, 60)
        return generate_pqrst_complex(hr, sampling_rate, duration, noise_level=0.05)
    
    elif condition == 'arrhythmia':
        # Irregular heart rhythm
        n_samples = int(duration * sampling_rate)
        signal = np.zeros(n_samples)
        t = np.linspace(0, duration, n_samples)
        
        # Random beat intervals
        beat_times = [0]
        current_time = 0
        while current_time < duration:
            # Variable interval between beats
            interval = np.random.uniform(0.4, 0.9)
            current_time += interval
            if current_time < duration:
                beat_times.append(current_time)
        
        # Generate beats at irregular intervals
        for beat_time in beat_times:
            waves = {
                'P': (0.16, 0.25, 0.02),
                'Q': (0.20, -0.15, 0.01),
                'R': (0.22, 1.5, 0.015),
                'S': (0.24, -0.4, 0.01),
                'T': (0.38, 0.35, 0.04),
            }
            
            for wave_name, (t_offset, amplitude, width) in waves.items():
                wave_center = beat_time + t_offset
                wave = amplitude * np.exp(-((t - wave_center) ** 2) / (2 * width ** 2))
                signal += wave
        
        signal += 0.1 * np.random.randn(n_samples)
        return signal
    
    else:
        return generate_pqrst_complex(70, sampling_rate, duration)

def create_dataset(n_samples=1000, window_size=250, normal_ratio=0.5):
    """
    Create balanced dataset of normal and abnormal EKG signals.
    
    Parameters:
    -----------
    n_samples : int
        Total number of samples
    window_size : int
        Length of each signal window
    normal_ratio : float
        Proportion of normal samples
    
    Returns:
    --------
    X : numpy.ndarray
        Feature matrix (n_samples, window_size)
    y : numpy.ndarray
        Labels (0: normal, 1: tachycardia, 2: bradycardia, 3: arrhythmia)
    """
    n_normal = int(n_samples * normal_ratio)
    n_abnormal = n_samples - n_normal
    n_per_condition = n_abnormal // 3
    
    X = np.zeros((n_samples, window_size))
    y = np.zeros(n_samples, dtype=int)
    
    idx = 0
    
    # Generate normal samples
    for _ in range(n_normal):
        hr = np.random.randint(60, 100)
        X[idx] = generate_pqrst_complex(hr, window_size, 1.0, 0.05)
        y[idx] = 0  # Normal
        idx += 1
    
    # Generate abnormal samples
    conditions = ['tachycardia', 'bradycardia', 'arrhythmia']
    for i, condition in enumerate(conditions):
        for _ in range(n_per_condition):
            X[idx] = generate_abnormal_ekg(condition, window_size, 1.0)
            y[idx] = i + 1
            idx += 1
    
    # Shuffle dataset
    shuffle_idx = np.random.permutation(n_samples)
    X = X[shuffle_idx]
    y = y[shuffle_idx]
    
    return X, y

## BPNN Implementation

In [ ]:
class ActivationFunctions:
    """Collection of activation functions and their derivatives."""
    
    @staticmethod
    def sigmoid(z):
        """Sigmoid: σ(z) = 1 / (1 + e^(-z))"""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    @staticmethod
    def sigmoid_derivative(z):
        """Sigmoid derivative: σ'(z) = σ(z)(1 - σ(z))"""
        s = ActivationFunctions.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def tanh(z):
        """Hyperbolic tangent: tanh(z) = (e^z - e^(-z)) / (e^z + e^(-z))"""
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        """Tanh derivative: tanh'(z) = 1 - tanh²(z)"""
        t = np.tanh(z)
        return 1 - t ** 2
    
    @staticmethod
    def relu(z):
        """ReLU: max(0, z)"""
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        """ReLU derivative: 1 if z > 0, else 0"""
        return (z > 0).astype(float)
    
    @staticmethod
    def softmax(z):
        """Softmax: exp(z_i) / Σ(exp(z_j))"""
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)

class BPNN:
    """
    Backpropagation Neural Network implementation.
    
    Mathematical formulations:
    - Forward: z^[l] = W^[l] * a^[l-1] + b^[l]
    - Activation: a^[l] = g(z^[l])
    - Loss: L = (1/m) * Σ loss_function(y, ŷ)
    - Backprop: δ^[l] = (W^[l+1])^T * δ^[l+1] ⊙ g'(z^[l])
    - Update: W^[l] := W^[l] - α * dW^[l]
    """
    
    def __init__(self, layer_dims, activation='relu', learning_rate=0.01):
        """
        Initialize BPNN.
        
        Parameters:
        -----------
        layer_dims : list
            List of layer dimensions [input_dim, hidden1, hidden2, ..., output_dim]
        activation : str
            Activation function ('sigmoid', 'tanh', 'relu')
        learning_rate : float
            Learning rate α
        """
        self.layer_dims = layer_dims
        self.learning_rate = learning_rate
        self.activation_name = activation
        
        # Set activation function
        if activation == 'sigmoid':
            self.activation = ActivationFunctions.sigmoid
            self.activation_derivative = ActivationFunctions.sigmoid_derivative
        elif activation == 'tanh':
            self.activation = ActivationFunctions.tanh
            self.activation_derivative = ActivationFunctions.tanh_derivative
        else:  # relu
            self.activation = ActivationFunctions.relu
            self.activation_derivative = ActivationFunctions.relu_derivative
        
        # Initialize parameters using He initialization
        self.parameters = {}
        self._initialize_parameters()
        
        # Cache for forward propagation
        self.cache = {}
        
        # Training history
        self.history = {'loss': [], 'accuracy': []}
    
    def _initialize_parameters(self):
        """
        Initialize weights and biases using He initialization:
        W^[l] ~ N(0, sqrt(2/n^[l-1]))
        b^[l] = 0
        """
        np.random.seed(42)
        L = len(self.layer_dims)
        
        for l in range(1, L):
            # He initialization: W ~ N(0, sqrt(2/n_in))
            self.parameters[f'W{l}'] = np.random.randn(
                self.layer_dims[l], self.layer_dims[l-1]
            ) * np.sqrt(2.0 / self.layer_dims[l-1])
            
            # Initialize biases to zero
            self.parameters[f'b{l}'] = np.zeros((self.layer_dims[l], 1))
    
    def forward_propagation(self, X):
        """
        Forward propagation through the network.
        
        For each layer l:
        z^[l] = W^[l] * a^[l-1] + b^[l]
        a^[l] = g(z^[l])
        
        Parameters:
        -----------
        X : numpy.ndarray
            Input data (n_features, m_samples)
        
        Returns:
        --------
        AL : numpy.ndarray
            Output activations
        """
        L = len(self.layer_dims) - 1
        A = X
        self.cache['A0'] = X
        
        # Hidden layers
        for l in range(1, L):
            A_prev = A
            W = self.parameters[f'W{l}']
            b = self.parameters[f'b{l}']
            
            # Linear transformation: z = W * a + b
            Z = np.dot(W, A_prev) + b
            
            # Activation: a = g(z)
            A = self.activation(Z)
            
            # Cache for backpropagation
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A
        
        # Output layer (softmax)
        W = self.parameters[f'W{L}']
        b = self.parameters[f'b{L}']
        Z = np.dot(W, A) + b
        AL = ActivationFunctions.softmax(Z)
        
        self.cache[f'Z{L}'] = Z
        self.cache[f'A{L}'] = AL
        
        return AL
    
    def compute_loss(self, AL, Y):
        """
        Compute categorical cross-entropy loss:
        L = -(1/m) * Σ Σ y_k * log(ŷ_k)
        
        Parameters:
        -----------
        AL : numpy.ndarray
            Predicted probabilities
        Y : numpy.ndarray
            True labels (one-hot encoded)
        
        Returns:
        --------
        loss : float
            Cross-entropy loss
        """
        m = Y.shape[1]
        
        # Categorical cross-entropy: -(1/m) * Σ y * log(ŷ)
        loss = -np.sum(Y * np.log(AL + 1e-8)) / m
        
        return loss
    
    def backward_propagation(self, Y):
        """
        Backward propagation to compute gradients.
        
        Output layer: δ^[L] = a^[L] - y
        Hidden layers: δ^[l] = (W^[l+1])^T * δ^[l+1] ⊙ g'(z^[l])
        
        Weight gradient: dW^[l] = (1/m) * δ^[l] * (a^[l-1])^T
        Bias gradient: db^[l] = (1/m) * Σ δ^[l]
        
        Parameters:
        -----------
        Y : numpy.ndarray
            True labels (one-hot encoded)
        
        Returns:
        --------
        gradients : dict
            Dictionary of gradients for weights and biases
        """
        m = Y.shape[1]
        L = len(self.layer_dims) - 1
        gradients = {}
        
        # Output layer gradient: δ^[L] = a^[L] - y
        dZ = self.cache[f'A{L}'] - Y
        
        # Backpropagate through layers
        for l in reversed(range(1, L + 1)):
            A_prev = self.cache[f'A{l-1}']
            
            # Weight gradient: dW = (1/m) * dZ * A_prev^T
            gradients[f'dW{l}'] = np.dot(dZ, A_prev.T) / m
            
            # Bias gradient: db = (1/m) * Σ dZ
            gradients[f'db{l}'] = np.sum(dZ, axis=1, keepdims=True) / m
            
            if l > 1:
                # Hidden layer gradient: dZ = W^T * dZ ⊙ g'(Z)
                W = self.parameters[f'W{l}']
                Z_prev = self.cache[f'Z{l-1}']
                dZ = np.dot(W.T, dZ) * self.activation_derivative(Z_prev)
        
        return gradients
    
    def update_parameters(self, gradients):
        """
        Update parameters using gradient descent:
        W^[l] := W^[l] - α * dW^[l]
        b^[l] := b^[l] - α * db^[l]
        
        Parameters:
        -----------
        gradients : dict
            Dictionary of gradients
        """
        L = len(self.layer_dims) - 1
        
        for l in range(1, L + 1):
            # Gradient descent update
            self.parameters[f'W{l}'] -= self.learning_rate * gradients[f'dW{l}']
            self.parameters[f'b{l}'] -= self.learning_rate * gradients[f'db{l}']
    
    def train(self, X, y, epochs=1000, batch_size=32, verbose=True):
        """
        Train the neural network.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data (m_samples, n_features)
        y : numpy.ndarray
            Labels
        epochs : int
            Number of training epochs
        batch_size : int
            Mini-batch size
        verbose : bool
            Print training progress
        """
        m = X.shape[0]
        n_batches = m // batch_size
        
        # One-hot encode labels
        n_classes = len(np.unique(y))
        Y = np.eye(n_classes)[y].T
        X = X.T
        
        for epoch in range(epochs):
            # Shuffle data
            permutation = np.random.permutation(m)
            X_shuffled = X[:, permutation]
            Y_shuffled = Y[:, permutation]
            
            epoch_loss = 0
            
            # Mini-batch gradient descent
            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size
                
                X_batch = X_shuffled[:, start:end]
                Y_batch = Y_shuffled[:, start:end]
                
                # Forward propagation
                AL = self.forward_propagation(X_batch)
                
                # Compute loss
                loss = self.compute_loss(AL, Y_batch)
                epoch_loss += loss
                
                # Backward propagation
                gradients = self.backward_propagation(Y_batch)
                
                # Update parameters
                self.update_parameters(gradients)
            
            # Compute accuracy
            AL = self.forward_propagation(X)
            predictions = np.argmax(AL, axis=0)
            accuracy = np.mean(predictions == y)
            
            # Store history
            avg_loss = epoch_loss / n_batches
            self.history['loss'].append(avg_loss)
            self.history['accuracy'].append(accuracy)
            
            # Print progress
            if verbose and (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f} - Accuracy: {accuracy:.4f}")
    
    def predict(self, X):
        """
        Make predictions.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Input data (m_samples, n_features)
        
        Returns:
        --------
        predictions : numpy.ndarray
            Predicted class labels
        """
        AL = self.forward_propagation(X.T)
        predictions = np.argmax(AL, axis=0)
        return predictions
    
    def predict_proba(self, X):
        """
        Predict class probabilities.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Input data (m_samples, n_features)
        
        Returns:
        --------
        probabilities : numpy.ndarray
            Class probabilities (m_samples, n_classes)
        """
        AL = self.forward_propagation(X.T)
        return AL.T

## Example: Training BPNN on EKG Data

In [ ]:
# Generate synthetic dataset
print("Generating synthetic EKG dataset...")
X, y = create_dataset(n_samples=1000, window_size=250, normal_ratio=0.5)

print(f"Dataset shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Classes: {np.unique(y)} (0: Normal, 1: Tachycardia, 2: Bradycardia, 3: Arrhythmia)")

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (z-score normalization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Initialize BPNN
# Architecture: 250 (input) -> 128 -> 64 -> 32 -> 4 (output classes)
layer_dims = [250, 128, 64, 32, 4]
bpnn = BPNN(layer_dims=layer_dims, activation='relu', learning_rate=0.01)

print("BPNN Architecture:")
for i, dim in enumerate(layer_dims):
    if i == 0:
        print(f"  Input layer: {dim} neurons")
    elif i == len(layer_dims) - 1:
        print(f"  Output layer: {dim} neurons (softmax)")
    else:
        print(f"  Hidden layer {i}: {dim} neurons ({bpnn.activation_name})")

# Count parameters
total_params = 0
for l in range(1, len(layer_dims)):
    n_weights = layer_dims[l] * layer_dims[l-1]
    n_biases = layer_dims[l]
    total_params += n_weights + n_biases
    print(f"  Layer {l}: {n_weights} weights + {n_biases} biases = {n_weights + n_biases} parameters")

print(f"\nTotal trainable parameters: {total_params}")

In [ ]:
# Train the model
print("\nTraining BPNN...")
bpnn.train(X_train, y_train, epochs=500, batch_size=32, verbose=True)

In [ ]:
# Evaluate on test set
y_pred = bpnn.predict(X_test)
test_accuracy = np.mean(y_pred == y_test)

print(f"\nTest Accuracy: {test_accuracy:.4f}")

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

class_names = ['Normal', 'Tachycardia', 'Bradycardia', 'Arrhythmia']
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

## Visualize Training Progress

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Loss curve
ax1.plot(bpnn.history['loss'])
ax1.set_title('Training Loss Over Time')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True)

# Accuracy curve
ax2.plot(bpnn.history['accuracy'])
ax2.set_title('Training Accuracy Over Time')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.grid(True)

plt.tight_layout()
plt.show()

## Visualize Sample Predictions

In [ ]:
# Visualize sample predictions
n_samples_to_show = 8
sample_indices = np.random.choice(len(X_test), n_samples_to_show, replace=False)

fig, axes = plt.subplots(2, 4, figsize=(20, 8))
axes = axes.ravel()

for i, idx in enumerate(sample_indices):
    signal = X_test[idx]
    true_label = class_names[y_test[idx]]
    pred_label = class_names[y_pred[idx]]
    
    # Get prediction probabilities
    proba = bpnn.predict_proba(X_test[idx:idx+1])[0]
    confidence = np.max(proba)
    
    axes[i].plot(signal)
    axes[i].set_title(f'True: {true_label}\nPred: {pred_label} ({confidence:.2f})')
    axes[i].set_xlabel('Sample')
    axes[i].set_ylabel('Amplitude')
    
    # Color code: green if correct, red if incorrect
    color = 'green' if true_label == pred_label else 'red'
    axes[i].spines['top'].set_color(color)
    axes[i].spines['bottom'].set_color(color)
    axes[i].spines['left'].set_color(color)
    axes[i].spines['right'].set_color(color)
    axes[i].spines['top'].set_linewidth(3)
    axes[i].spines['bottom'].set_linewidth(3)
    axes[i].spines['left'].set_linewidth(3)
    axes[i].spines['right'].set_linewidth(3)

plt.tight_layout()
plt.show()

## Summary of Mathematical Formulations

This notebook implements a complete BPNN with the following mathematical expressions:

### 1. Forward Propagation
- **Linear transformation**: $z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]}$
- **Activation**: $a^{[l]} = g(z^{[l]})$ where $g$ can be sigmoid, tanh, or ReLU
- **Output layer**: $a^{[L]} = \text{softmax}(z^{[L]})$

### 2. Loss Computation
- **Categorical cross-entropy**: $L = -\frac{1}{m} \sum_{i=1}^{m} \sum_{k=1}^{K} y_k^{(i)} \log(\hat{y}_k^{(i)})$

### 3. Backpropagation
- **Output gradient**: $\delta^{[L]} = a^{[L]} - y$
- **Hidden gradients**: $\delta^{[l]} = (W^{[l+1]})^T \delta^{[l+1]} \odot g'^{[l]}(z^{[l]})$
- **Weight gradients**: $\frac{\partial L}{\partial W^{[l]}} = \frac{1}{m} \delta^{[l]} (a^{[l-1]})^T$
- **Bias gradients**: $\frac{\partial L}{\partial b^{[l]}} = \frac{1}{m} \sum_{i=1}^{m} \delta^{[l]}_i$

### 4. Parameter Updates
- **Gradient descent**: 
  - $W^{[l]} := W^{[l]} - \alpha \frac{\partial L}{\partial W^{[l]}}$
  - $b^{[l]} := b^{[l]} - \alpha \frac{\partial L}{\partial b^{[l]}}$

### 5. Activation Functions
- **Sigmoid**: $\sigma(z) = \frac{1}{1 + e^{-z}}$ with derivative $\sigma'(z) = \sigma(z)(1 - \sigma(z))$
- **Tanh**: $\tanh(z) = \frac{e^z - e^{-z}}{e^z + e^{-z}}$ with derivative $\tanh'(z) = 1 - \tanh^2(z)$
- **ReLU**: $\text{ReLU}(z) = \max(0, z)$ with derivative $\text{ReLU}'(z) = \mathbb{1}_{z > 0}$
- **Softmax**: $\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}$

All mathematical expressions have been implemented and validated in this notebook.